In [1]:
from datetime import datetime

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

from logger import Logger
from solver import Solver
from crossngover import CrossN

In [14]:
#!pip install tqdm

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=216,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=216,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [54]:
# net = torchvision.models.mobilenet_v2(pretrained=True)
# num_ftrs = net.last_channel

# classes = ('plane', 'car', 'bird', 'cat',
#         'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# net.classifier = nn.Linear(num_ftrs, len(classes))
# net.cuda()
# print()

In [3]:
net = torchvision.models.resnet101(pretrained=True)
net.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [4]:
num_ftrs = 1000

classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

net.classifier = nn.Linear(num_ftrs, len(classes))
net.cuda()
print()

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [7]:
!nvidia-smi

/bin/sh: 1: nvidia-smi: not found


mobilenet from 0:
- Evo + Cross  - DONE
- Evo Only - DONE
- Standard - DONE

resnet from 0:
- Evo + Cross - DONE
- Evo Only - DONE
- Standard - DONE

mobilenet pretrained:
- Evo + Cross  - DONE
- Evo Only   - DONE
- Standard - DONE

resnet pretrained:
- Evo + Cross  - DONE
- Evo Only - DONE
- Standard - processing

In [66]:
experiment_note = """ 
NET: resnet
pretrained: True
Training mode: No Evo and CrossN
Optimizer SGD
lr = 0.001 
momentum = 0.9
epochs = 100
Params: Default"""

logger = Logger('train', experiment_note)

In [67]:
logger.path

'./experiments/2019-12-15/train_12:47'

In [7]:
# post_note = "Test After Experiment info"
# logger.add_post_result(post_note)

In [68]:
lr = 0.001

optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()
evo_optim = CrossN()

def validation(net, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return float(100 * correct / total)

In [69]:
solver = Solver(
    net,
    optimizer,
    logger,
    criterion,
    validation,
    evo_optim, 
    trainloader,
    testloader,
    testloader,  
    epochs=100,
    evo_step=100,
    child_count=20,
    best_child_count=3,
    mode='normal',
    debug=False,
    lr=lr)

In [70]:
logger.add_post_result(f'start: {datetime.now()}')
solver.start()
logger.add_post_result(f'finish: {datetime.now()}')
logger.close()

Start training
Epoch: 0


100%|██████████| 19/19 [01:48<00:00,  5.73s/it]

best child - 0.0
Epoch: 1


[2] loss: 1.042 validation score: 70.55 %
Epoch: 2
[3] loss: 0.421 validation score: 74.79 %
Epoch: 3
[4] loss: 0.122 validation score: 75.34 %
Epoch: 4
[5] loss: 0.020 validation score: 75.04 %
Epoch: 5
[6] loss: 0.028 validation score: 75.12 %
Epoch: 6
[7] loss: 0.007 validation score: 75.63 %
Epoch: 7
[8] loss: 0.002 validation score: 75.94 %
Epoch: 8
[9] loss: 0.001 validation score: 76.11 %
Epoch: 9
[10] loss: 0.001 validation score: 76.43 %
Epoch: 10
[11] loss: 0.000 validation score: 76.25 %
Epoch: 11
[12] loss: 0.000 validation score: 76.43 %
Epoch: 12
[13] loss: 0.000 validation score: 76.40 %
Epoch: 13
[14] loss: 0.001 validation score: 76.36 %
Epoch: 14
[15] loss: 0.000 validation score: 76.32 %
Epoch: 15
[16] loss: 0.000 validation score: 76.21 %
Epoch: 16
[17] loss: 0.000 validation score: 76.20 %
Epoch: 17
[18] loss: 0.000 validation score: 76.19 %
Epoch: 18
[19] loss: 0.000 validation score: 76.23 %
Epoch: 19
[20] loss: 0.000 validation score: 76.22 %
Epoch: 20
[21] loss

In [25]:
logger.close()